In [4]:
import pandas as  pd
from sqlalchemy import create_engine, text
import psycopg2
from configparser import ConfigParser
import datetime
import os
from weather_etl.utils import read_config

/home/bedo7/weather_de/weather_etl/extract.py


In [ ]:
parser = ConfigParser()
read_config(parser)
print(50*"&")
print(f"current path {os.getcwd()}")
print(50*"&")

# Load database configuration
dbname = parser.get("postgres_config", "database")
user = parser.get("postgres_config", "username")
password = parser.get("postgres_config", "password")
host = parser.get("postgres_config", "host")
port = parser.get("postgres_config", "port")

# Load dataset configuration
path = parser.get("pathes", "transformed")
print(path)
# Read the data in pandas
df = pd.read_csv(path)


# make Connection
DATABASE_URL = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}"

engine = create_engine(DATABASE_URL, echo=True)

raw_conn = engine.raw_connection()







# pd.read_sql("select * from weather", con=conn)

# # Get the last updated record
# # query = text("select max(timestamp) from weather")
# # conn.execute(query)
# # maximum_timestamp = int(conn.execute(query).fetchone()[0]) or 0 

# # Select the new data to insert on the database
# # df = df.query(f"timestamp > {maximum_timestamp}")
# print(df)



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
current path /home/bedo7/weather_de/weather_etl
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
../data/weather_transformed.csv
2025-02-22 16:08:54,492 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-02-22 16:08:54,492 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-22 16:08:54,494 INFO sqlalchemy.engine.Engine select current_schema()
2025-02-22 16:08:54,494 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-22 16:08:54,495 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-02-22 16:08:54,495 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-22 16:08:54,503 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-22 16:08:54,509 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY

803

In [7]:
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS my_table
    (
        id SERIAL PRIMARY KEY,
        name TEXT NOT NULL,
        created_at TIMESTAMP DEFAULT NOW()
    );
    """
)
conn.commit()
cursor.close()
conn.close()

## Read data from database with pandas

In [10]:
import pandas as pd 
import psycopg2
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
query = "select * from my_table"
df = pd.read_sql(query, conn)
conn.close()

/tmp/ipykernel_1367/1444886856.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [41]:
import psycopg2
import pandas as pd

try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    with conn:
        with conn.cursor() as cursor:
            cursor.execute("CREATE SCHEMA IF NOT EXISTS test_schema;")
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS test_schema.test_table (
                    id SERIAL PRIMARY KEY,
                    name TEXT NOT NULL,
                    created_at TIMESTAMP DEFAULT NOW()
                );
            """)
            cursor.execute("""
                INSERT INTO test_schema.test_table (name)
                VALUES ('bedo'), ('gpto3'), ('notbedo'), ('last_test_name');
            """)
    df = pd.read_sql("SELECT * FROM test_schema.test_table;", conn)
    print(df)
except Exception as e:
    print("An error occurred:", e)
finally:
    if conn:
        conn.close()


An error occurred: duplicate key value violates unique constraint "test_table_pkey"
DETAIL:  Key (id)=(1) already exists.



In [39]:
import pandas as pd
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cursor = conn.cursor()
query = "select * from test_schema.test_table;"

pd.read_sql(query, conn)

/tmp/ipykernel_1367/1554634624.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,id,name,created_at
0,1,bedo,2025-02-22 14:45:46.101218
1,2,gpto3,2025-02-22 14:45:46.101218
2,3,notbedo,2025-02-22 14:45:46.101218
3,4,last_test_name,2025-02-22 14:45:46.101218


In [48]:
DATABASE_URL = f"postgresql://{user}:{password}@{host}/{dbname}"
engine = create_engine(DATABASE_URL, echo=True)

connection = engine.raw_connection()
try:
    df = pd.read_sql('SELECT * FROM test_schema.test_table', connection)
    print(df)
finally:
    connection.close()


2025-02-22 15:00:16,017 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-02-22 15:00:16,019 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-22 15:00:16,021 INFO sqlalchemy.engine.Engine select current_schema()
2025-02-22 15:00:16,023 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-22 15:00:16,025 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-02-22 15:00:16,027 INFO sqlalchemy.engine.Engine [raw sql] {}


   id            name                 created_at
0   1            bedo 2025-02-22 14:45:46.101218
1   2           gpto3 2025-02-22 14:45:46.101218
2   3         notbedo 2025-02-22 14:45:46.101218
3   4  last_test_name 2025-02-22 14:45:46.101218


/tmp/ipykernel_1367/3216653403.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM test_schema.test_table', connection)
